# Template: Минимальная нейросеть PyTorch для **любой** бинарной классификации

**Формат:** весь код с пропусками `_____` + краткие подсказки.  
**Цель:** чтобы вы могли взять **любой CSV с Kaggle**, указать `TARGET_COL`, и получить:
- preprocessing (числа + категории)
- train/test split
- DataLoader
- простую MLP-модель
- train/eval loop + метрики
- подбор порога
- сохранение/загрузка модели

> Заполняйте пропуски `_____` сверху вниз, по порядку.


## 0) Что нужно от вашего датасета
1) CSV файл (например `train.csv`)  
2) Название целевой колонки: `TARGET_COL` (в ней 0/1 или True/False)  
3) Остальные колонки — признаки (числовые и/или категориальные)

Если у вас target не 0/1, приведите к 0/1:
- `df[TARGET_COL] = (df[TARGET_COL] == "Yes").astype(int)`
- `df[TARGET_COL] = df[TARGET_COL].astype(int)` (для True/False)


In [ ]:
# ====== 0) Imports ======
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("torch:", torch.__version__)
print("device:", device)

# 1) Загрузка данных

**Подсказка:** у Kaggle обычно `train.csv` лежит в папке проекта.  
Если вы в Colab, сначала загрузите файл в окружение или подключите Google Drive.

Заполните:
- `DATA_PATH` — путь к CSV
- `TARGET_COL` — название целевой колонки


In [ ]:
# ====== 1) Load CSV ======
DATA_PATH = "_____"          # например: "train.csv" или "/content/train.csv"
TARGET_COL = "_____"         # например: "Survived" / "target" / "is_fraud"

df = pd.read_csv(_____)
print("shape:", df._____)
df.head()

## 1.1) Быстрая проверка target

**Подсказка:** target должен быть бинарным.  
Если target строковый/True-False — приведите к 0/1.


In [ ]:
# ====== 1.1) Target sanity check ======
print(df[TARGET_COL].value_counts(dropna=False).head(10))

# Примеры преобразований (исправьте под свой датасет):
# df[TARGET_COL] = (df[TARGET_COL] == "_____").astype(int)
# df[TARGET_COL] = df[TARGET_COL].astype(int)

# 2) Features/Target + train/test split

Заполните:
- `TEST_SIZE` (обычно 0.2 или 0.25)
- `RANDOM_STATE`


In [ ]:
# ====== 2) Split ======
TEST_SIZE = _____
RANDOM_STATE = _____

X = df.drop(columns=[_____])
y = df[TARGET_COL].astype(np.float32).values  # (n,)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=_____, random_state=_____, stratify=y
)

print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train mean:", y_train._____(), "y_test mean:", y_test._____())

# 3) Определяем числовые и категориальные колонки

**Подсказка:** обычно:
- числовые: `int/float`
- категориальные: `object/category/bool`

Если у вас есть ID-колонка (например `PassengerId`), её можно удалить:
- `X = X.drop(columns=['PassengerId'])`


In [ ]:
# ====== 3) Column lists ======
num_features = X_train.select_dtypes(include=["int64","int32","float64","float32"]).columns.tolist()
cat_features = X_train.select_dtypes(include=["object","category","bool"]).columns.tolist()

print("num_features:", len(_____))
print("cat_features:", len(_____))
print("Example num:", _____[:10])
print("Example cat:", _____[:10])

# 4) Preprocessing pipeline (важно против data leakage)

- Числовые: `SimpleImputer(median)` + `StandardScaler()`
- Категориальные: `SimpleImputer(most_frequent)` + `OneHotEncoder(handle_unknown='ignore')`

В результате получаем `X_train_proc`, `X_test_proc`.


In [ ]:
# ====== 4) Preprocess with ColumnTransformer ======
numeric_pipe = Pipeline(steps=[
    ("imputer", _____(strategy="_____")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="_____")),
    ("ohe", One_____(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(transformers=[
    ("num", _____, num_features),
    ("cat", categorical_pipe, _____),
])

X_train_proc = preprocess.fit_transform(_____)
X_test_proc  = preprocess.transform(_____)

print("Processed train shape:", X_train_proc.shape)
print("Processed test  shape:", X_test_proc.shape)

# 5) Torch tensors + DataLoader

`ColumnTransformer` часто возвращает sparse matrix. Для учебных датасетов можно `.toarray()`.

Заполните:
- `BATCH_SIZE`
- `TEST_BATCH`


In [ ]:
# ====== 5) To torch ======
X_train_dense = X_train_proc.toarray() if hasattr(X_train_proc, "toarray") else X_train_proc
X_test_dense  = X_test_proc.toarray() if hasattr(X_test_proc, "toarray")  else X_test_proc

X_train_t = torch.tensor(_____, dtype=torch.float32)
X_test_t = torch.tensor(_____, dtype=torch.float32)

y_train_t = torch.tensor(y_train.reshape(-1,1), dtype=torch.float32)
y_test_t = torch.tensor(y_test.reshape(-1,1), dtype=torch.float32)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

BATCH_SIZE = _____
TEST_BATCH = _____

train_loader = DataLoader(_____, batch_size=_____, shuffle=True)
test_loader  = DataLoader(_____,  batch_size=_____, shuffle=False)

xb, yb = next(iter(train_loader))
print("batch X:", xb.shape, xb.dtype)
print("batch y:", yb.shape, yb.dtype)

# 6) Модель: минимальный MLP

Заполните:
- `HIDDEN` (например 64)
- `DROPOUT` (например 0.2 или 0.0)

`INPUT_DIM` берём автоматически из `X_train_t.shape[1]`.


In [ ]:
# ====== 6) Model ======
INPUT_DIM = X_train_t.shape[1]
HIDDEN = _____
DROPOUT = _____

class KaggleBinaryMLP(nn.Module):
    def __init__(self, input_dim, hidden=_____, dropout=_____):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden)
        self.drop = nn.Dropout(_____)
        self.fc2 = nn.Linear(hidden, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.drop(x)
        logits = self.fc2(x)
        return logits

model = KaggleBinaryMLP(INPUT_DIM, hidden=HIDDEN, dropout=DROPOUT).to(_____)
print(model)
print("params:", sum(p.numel() for p in model.parameters()))

# 7) Loss + Optimizer

Заполните:
- `LR`


In [ ]:
# ====== 7) Loss & Optimizer ======
criterion = nn.BCEWithLogitsLoss()
LR = _____
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print("criterion:", type(_____).__name__)
print("optimizer:", type(_____).__name__, "lr:", LR)

# 8) Train/Eval loop

Заполните:
- `EPOCHS`


In [ ]:
# ====== 8) Train/Eval ======
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0

    for Xb, yb in loader:
        Xb = Xb.to(_____)
        yb = yb.to(_____)

        logits = model(Xb)
        loss = criterion(logits, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * Xb.size(0)

    return total_loss / len(loader.dataset)

@torch.no_grad()
def predict_probs(model, loader, device):
    model.eval()
    probs_all, y_all = [], []
    for Xb, yb in loader:
        Xb = Xb.to(device)
        logits = model(Xb)
        probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
        probs_all.append(probs)
        y_all.append(yb.numpy().reshape(-1))
    return np.concatenate(probs_all), np.concatenate(y_all)

@torch.no_grad()
def evaluate(model, loader, device, threshold=0.5):
    probs, y_true = predict_probs(model, loader, device)
    y_true = y_true.astype(int)
    y_pred = (probs >= threshold).astype(int)

    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    try:
        auc = roc_auc_score(y_true, probs)
    except Exception:
        auc = None
    return acc, cm, auc, probs, y_true, y_pred

EPOCHS = _____
history = []
for epoch in range(1, EPOCHS+1):
    loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    acc, _, auc, *_ = evaluate(model, test_loader, device, threshold=0.5)
    history.append((epoch, loss, acc, auc if auc is not None else np.nan))

    if epoch % 5 == 0 or epoch == 1:
        print(f"epoch={epoch:03d} loss={loss:.4f} acc={acc:.4f} auc={auc}")

hist = pd.DataFrame(history, columns=["epoch","loss","acc","auc"])
hist.tail()

In [ ]:
# Графики
plt.figure(figsize=(6,4))
plt.plot(_____["epoch"], _____["loss"])
plt.title("Loss")
plt.xlabel("epoch"); plt.ylabel("loss")
plt.grid(True); plt.show()

plt.figure(figsize=(6,4))
plt.plot(_____["epoch"], _____["acc"])
plt.title("Test accuracy")
plt.xlabel("epoch"); plt.ylabel("acc")
plt.grid(True); plt.show()

# 9) Подбор порога (threshold)

Заполните `BEST_THR`.


In [ ]:
# ====== 9) Threshold ======
for thr in [0.3, 0.5, 0.7]:
    acc, cm, auc, probs, y_true, y_pred = evaluate(model, test_loader, device, threshold=thr)
    print(f"thr={thr}: acc={acc:.4f} auc={auc}")
    print("CM:\n", cm)
    print("-"*40)

BEST_THR = _____
acc, cm, auc, probs, y_true, y_pred = evaluate(model, test_loader, device, threshold=BEST_THR)
print("BEST_THR:", BEST_THR, "acc:", _____, "auc:", _____)
print("CM:\n", _____)
print(classification_report(_____, _____, digits=3))

# 10) Сохранение и загрузка (model + preprocess)

Сохраняем:
- preprocess через joblib
- веса модели через torch.save(state_dict)


In [ ]:
# ====== 10) Save/Load ======
import joblib

PREPROCESS_PATH = "/preprocess.joblib"
MODEL_PATH = "/my_kaggle_dict.pt"

joblib.dump(preprocess, _____)
torch.save(model.state_dict(), _____)

print("Saved preprocess:", PREPROCESS_PATH)
print("Saved model     :", MODEL_PATH)

# ---- Load example ----
preprocess2 = joblib.load(PREPROCESS_PATH)

model2 = KaggleBinaryMLP(INPUT_DIM, hidden=HIDDEN, dropout=DROPOUT).to(_____)
model2.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model2.eval()

acc2, cm2, auc2, *_ = evaluate(model2, test_loader, device, threshold=BEST_THR)
print("Loaded acc:", _____, "auc:", _____)
print("Loaded CM:\n", _____)

# 11) Inference / Submission

Если у Kaggle есть `test.csv` без target:
1) читаем `test.csv`
2) preprocess.transform
3) sigmoid → prob
4) threshold → 0/1
5) сохраняем submission.csv

Заполните:
- `TEST_PATH`
- `ID_COL`


In [ ]:
# ====== 11) Inference / Submission ======
TEST_PATH = "_____"          # "test.csv"
ID_COL = "_____"             # "id" / "PassengerId" / ...

SUBMISSION_PATH = "/submission.csv"

df_test = pd.read_csv(TEST_PATH)

# Если ID_COL нет:
# df_test[ID_COL] = np.arange(len(df_test))

X_new = df_test.copy()

X_new_proc = preprocess2.transform(_____)
X_new_dense = X_new_proc.toarray() if hasattr(X_new_proc, "toarray") else X_new_proc
X_new_t = torch.tensor(X_new_dense, dtype=torch.float32).to(device)

with torch.no_grad():
    logits = model2(X_new_t)
    probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)

pred = (probs >= BEST_THR).astype(int)  # или probs, если Kaggle просит вероятность

submission = pd.DataFrame({
    ID_COL: df_test[ID_COL],
    TARGET_COL: pred
})

submission.to_csv(SUBMISSION_PATH, index=False)
print("Saved submission:", _____)
submission.head()

---
## Чеклист (если что-то падает)
- Проверь `TARGET_COL` и что он бинарный.
- Если есть `NaN` в target — сначала разберись с ними (обычно такие строки удаляют).
- Если много категорий и OHE огромный: уменьшай `HIDDEN`, увеличивай `BATCH_SIZE`, или начни с меньшего набора фич.
- Если дисбаланс: подбирай `BEST_THR`, смотри `classification_report`, и можно добавить `pos_weight`.
